In [73]:
%run ../Python/paths.py

ImportError: No module named 'fluidsim'

In [56]:
import pandas as pd

df_noise = pd.DataFrame(columns=pd_columns)
df_vort = pd.DataFrame(columns=pd_columns)
df_noise

In [57]:
df_paths = pd.DataFrame(index=list(paths_sim.keys()), data={'path': list(paths_sim.values())},)
df_paths.head()

,path
vortex_grid_c100nh1920Bu1.0efr1.00e+00,/media/avmo/lacie/13KTH/vortex_grid/SW1Lexlin_...
vortex_grid_c400nh1920Buinfefr1.00e+00,/media/avmo/lacie/13KTH/vortex_grid/SW1Lexlin_...
noise_c20nh1920Bu1.0,/media/avmo/lacie/13KTH/noise/SW1Lwaves_NOISE2...
vortex_grid_c20nh1920Buinfefr1.00e-01,/media/avmo/lacie/13KTH/vortex_grid/SW1Lexlin_...
noise_c20nh7680Buinf,/media/avmo/lacie/13KTH/noise/SW1Lwaves_NOISE2...


,$n$,$c$,$\nu_8$,$f$,$\epsilon$,$\frac{k_d}{k_f}$,$F_f$,$Ro_f$,$Bu$,$\min h$,$\frac{\max |\bf u|}{c}$


In [70]:
for key, row in df_paths.items:
    path = row[0]
    keyparams = keyparams_from_path(path)
    s = pandas_from_path(path, key)
    if keyparams[0] == 'noise':
        # df_noise = pd.concat(df_noise, df)
        df_noise = df_noise.append(s, ignore_index=True)
    else:
        # df_vort = pd.concat(df_vort, df)
        # df_vort.loc[len(df_vort.index)] = s
        df_vort = df_vort.append(s, ignore_index=True)

In [71]:
df_vort.head()

,$n$,$c$,$\nu_8$,$f$,$\epsilon$,$\frac{k_d}{k_f}$,$F_f$,$Ro_f$,$Bu$,$\min h$,$\frac{\max |\bf u|}{c}$
0,1920,100,1.111773e-12,75.398224,eps,kdkf,Fr,Ro,1.0,minh,maxuc
1,1920,400,1.111773e-12,0.000000,eps,kdkf,Fr,Ro,inf,minh,maxuc
2,1920,20,1.111773e-12,0.000000,eps,kdkf,Fr,Ro,inf,minh,maxuc
3,1920,20,1.111773e-12,7.539822,eps,kdkf,Fr,Ro,4.0,minh,maxuc
4,1920,20,1.111773e-12,0.000000,eps,kdkf,Fr,Ro,inf,minh,maxuc


In [72]:
df_noise.head()

,$n$,$c$,$\nu_8$,$f$,$\epsilon$,$\frac{k_d}{k_f}$,$F_f$,$Ro_f$,$Bu$,$\min h$,$\frac{\max |\bf u|}{c}$
0,1920,20,9.676842e-13,15.079645,eps,kdkf,Fr,Ro,1.0,minh,maxuc
1,7680,20,3.724795e-17,0.000000,eps,kdkf,Fr,Ro,inf,minh,maxuc
2,3840,400,6.006974e-15,0.000000,eps,kdkf,Fr,Ro,inf,minh,maxuc
3,960,40,1.560583e-10,0.000000,eps,kdkf,Fr,Ro,inf,minh,maxuc
4,7680,100,3.724795e-17,0.000000,eps,kdkf,Fr,Ro,inf,minh,maxuc


In [31]:
pdseries_from_path(path)

0            1920
1             400
2     1.11177e-12
3               0
4             eps
5            kdkf
6              Fr
7              Ro
8             inf
9            minh
10          maxuc
dtype: object

In [10]:
pd.concat?

Signature: pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, copy=True)
Docstring:
Concatenate pandas objects along a particular axis with optional set logic
along the other axes.

Can also add a layer of hierarchical indexing on the concatenation axis,
which may be useful if the labels are the same (or overlapping) on
the passed axis number.

Parameters
----------
objs : a sequence or mapping of Series, DataFrame, or Panel objects
    If a dict is passed, the sorted keys will be used as the `keys`
    argument, unless it is passed, in which case the values will be
    selected (see below). Any None objects will be dropped silently unless
    they are all None in which case a ValueError will be raised
axis : {0/'index', 1/'columns'}, default 0
    The axis to concatenate along
join : {'inner', 'outer'}, default 'outer'
    How to handle indexes on other axis(es)
join_axes : list of Index objects
    Spe